# Approach 2: Dataset Preparation with Prompt Completion Pairs

For the second dataset preparation approach, I processed data involving prompt completion pairs:

## Dataset Structure:

In this approach, the dataset structure was simplified to the following format:

```markdown
### Human:
### Companion:


In [8]:
import json

with open('./data/companion_base_dataset.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

new_data = []

for item in data:
    conversation_id = item['id']
    conversations = item['conversations']
    temp_list = []
    
    if len(item['conversations'])%2 == 0:
        for i in range(0,len(conversations),2):
            iteration = int(i/2)
            interated_conversations = f"conversation_{iteration}"
            temp_list.append(item['conversations'][i:i+2])
        new_item ={
            "id": conversation_id,
            "conversations": temp_list
        }
        new_data.append(new_item)
    else:
        print("Odd number of conversations")

output_file_path = './data/companion_dataset_v2.json'
with open(output_file_path, 'w') as output_file:
    json.dump(new_data, output_file, indent=2)

#### Editing Companion Name

In [9]:
import json

# Define your input
companion_name = "Avalon"

# Load the JSON content from the file
file_path = './data/companion_dataset_v2.json'  # Replace with your file path
with open(file_path, 'r', encoding='utf-8') as json_file:
    json_content = json_file.read()

# Replace the special keyword with your input
modified_content = json_content.replace("{CompanionLLama}", companion_name)

# Write the modified content back to the file
output_file_path = f'./data/{companion_name}_dataset.json'  # Replace with your desired output file path
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    output_file.write(modified_content)

print("Replacement complete. Modified JSON saved to:", output_file_path)

Replacement complete. Modified JSON saved to: ./data/Avalon_dataset.json


#### Convert to Text dataset to feed to the model

In [10]:
with open("./data/Avalon_dataset.json" , "r" , encoding="utf-8") as json_file:
    data = json.load(json_file)
    
formatted_conversations = []

for items in data:
    for conversations in items["conversations"]:
        conversation_length = len(conversations)
        text_conversation = ""
        for idx,role in enumerate(conversations):
            if role["from"] == "human":
                formatted_text = f"### Human: {role['value']}\n"
                text_conversation += formatted_text
            elif role["from"] == "gpt":
                if idx == conversation_length - 1:  # Check if it's the last element
                    formatted_text = f"### Response: {role['value']}"
                    text_conversation += formatted_text
                else:
                    formatted_text = f"### Companion: {role['value']}\n"
                    text_conversation += formatted_text
            
        formatted_conversations.append({"text":text_conversation})
        
output_file_path = './data/Avalon.json'
with open(output_file_path, 'w') as output_file:
    json.dump(formatted_conversations, output_file, indent=2)


## Push Dataset to Hub

In [ ]:
from huggingface_hub import notebook_login
from datasets import load_dataset
notebook_login()

In [ ]:
dataset = load_dataset('json', data_files='./data/Avalon.json' , split='train')

In [12]:
print(dataset['text'][:1])

["### Human: Hey Avalon, I've run into a bit of a tricky situation at work, and I'm not sure how to handle it. Do you have any advice?\n### Response: I'd be happy to help if I can. Can you give me some details about the situation you're facing?"]


In [13]:
print(len(dataset['text']))

29655


In [ ]:
dataset.push_to_hub("Avalon_instruction_30k")